<a href="https://colab.research.google.com/github/atkinsonsean/Capstone/blob/main/2_3_model_gpt2_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GPT-2 CHATBOT MODEL (MEDIUM)**

Initially I had not planned on running a medium version of GPT-2 because all my previous efforts ran into issues when I tried to run them on Google Colab.

Fortunately, by making tweaks to the GPU batch size, I was finally able to get it to work.

Nonetheless, this still took quite some time to run (about 20 hours).

I almost had a heart attack when the process shut down at the 18.5 hour mark. Luckily, I was able to continue it from a previously saved checkpoint.

Also, as noted in the notebook for the small version of my GPT-2 model, I owe a huge debt of gratitude to Hugging Face's [original project notebook](https://github.com/huggingface/transformers/blob/master/examples/legacy/run_language_modeling.py), [Nathan Cooper](https://nathancooper.io/i-am-a-nerd/chatbot/deep-learning/gpt2/2020/05/12/chatbot-part-1.html), and [Tornike Tsereteli](https://towardsdatascience.com/rick-and-morty-story-generation-with-gpt2-using-transformers-and-streamlit-in-57-lines-of-code-8f81a8f92692) whose earlier work guides and heavily influences my own.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
! pip -q install transformers

     |████████████████████████████████| 3.5 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 64.8 MB/s 
     |████████████████████████████████| 6.8 MB 81.3 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 89.7 MB/s 


# **IMPORTING LIBRARIES**

In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-medium")

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:807: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/823M [00:00<?, ?B/s]

# **GPT-2 OUT OF THE BOX TEST**

In [ ]:
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
# append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
# generated a response while limiting the total chat history to 1000 tokens    
    chat_history_ids = model.generate(
    bot_input_ids, max_length=1000,
    pad_token_id=tokenizer.eos_token_id
    )
# pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Tell me about Amazon
DialoGPT: I'm in the same boat. I'm in the same boat as you.
>> User:What do you know about Amazon?
DialoGPT: I'm in the same boat. I'm in the same boat.
>> User:What do you know about Microsoft?
DialoGPT: What do you know about Google?
>> User:What do you know about Google?
DialoGPT: What do you know about Google?
>> User:What is your name?
DialoGPT: What is your favorite thing about the internet?


# **CONFIGURING VARIABLES**

In [ ]:
class Args():
    def __init__(self):
        self.output_dir = 'output-medium-50-10-epochs'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-medium'
        self.config_name = 'microsoft/DialoGPT-medium'
        self.tokenizer_name = 'microsoft/DialoGPT-medium'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 1
        self.per_gpu_eval_batch_size = 1 #necessary to change this to 1 in order to get this to work on Google Colab
        self.gradient_accumulation_steps = 1 #necessary to change this to 1 in order to get this to work on Google Colab
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 10
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 5000
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = True
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

# **IMPORTING DATA**

In [ ]:
import pandas as pd

In [ ]:
topical_chat = pd.read_csv('topical_chat.csv')

The dataset I decided to use is a [topical chat dataset](https://www.kaggle.com/arnavsharmaas/chatbot-dataset-topical-chat) from Amazon that consists of 8,000+ conversations and 184,000+ messages.

It was created by Arnav Sharma and is a more streamlined version of [this](https://github.com/alexa/Topical-Chat) original Amazon Alexa dataset.

The dataset contains a conversation id, a message (which is either a reply or the start of a conversation), and the sentiment of each message.

The dataset spands 8 broad topics and contains conversation partner who do not have defined roles. It was created with the goal of [aiding in the effort to build a socialbot that can have deep, engaging open-domain conversations with humans](https://m.media-amazon.com/images/G/01/amazon.jobs/3079_Paper._CB1565131710_.pdf).

The eight broad topics are as follows:
- fashion
- politics
- books
- sports
- general entertainment
- music
- science and technology
- movies

In [ ]:
#due to the time it takes to train and fine-tune a model, I had to limit my data to the first 50,000 entries
df = topical_chat[:50000]

In [ ]:
df

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
49995,2289,So good that she broke 8 national records and...,Curious to dive deeper
49996,2289,Nice. Most players after retirement have fina...,Curious to dive deeper
49997,2289,Wow that is sad. DId you know Iverson signed ...,Curious to dive deeper
49998,2289,"I have not heard of it, did he get a good deal?",Curious to dive deeper


In [ ]:
df.head(50)

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
5,1,Google is leading the alphabet subsidiary and...,Curious to dive deeper
6,1,Did you know Google had hundreds of live goat...,Curious to dive deeper
7,1,"It is very interesting. Google provide ""Chrom...",Curious to dive deeper
8,1,I like Google Chrome. Do you use it as well f...,Curious to dive deeper
9,1,Yes.Google is the biggest search engine and G...,Curious to dive deeper


# **CREATING CONTEXT FOR EACH MESSAGE**

In [ ]:
contexted = []
n = 7
for i in range(n, len(df['message'])):
  row = []
  prev = i - 1 - n # we additionally subtract 1, so row will contain current response and 7 previous responses  
  for j in range(i, prev, -1):
    row.append(df['message'][j])
  contexted.append(row)
columns = ['response', 'context'] 
columns = columns + ['context/'+str(i) for i in range(n-1)]
df = pd.DataFrame.from_records(contexted, columns=columns)
df.head(5)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,"It is very interesting. Google provide ""Chrom...",Did you know Google had hundreds of live goat...,Google is leading the alphabet subsidiary and...,"Yeah, their services are good. I'm just not a...",Google provides online related services and p...,"I'm not a huge fan of Google, but I use it a...",Both are excellent technology they are helpfu...,Are you a fan of Google or Microsoft?
1,I like Google Chrome. Do you use it as well f...,"It is very interesting. Google provide ""Chrom...",Did you know Google had hundreds of live goat...,Google is leading the alphabet subsidiary and...,"Yeah, their services are good. I'm just not a...",Google provides online related services and p...,"I'm not a huge fan of Google, but I use it a...",Both are excellent technology they are helpfu...
2,Yes.Google is the biggest search engine and G...,I like Google Chrome. Do you use it as well f...,"It is very interesting. Google provide ""Chrom...",Did you know Google had hundreds of live goat...,Google is leading the alphabet subsidiary and...,"Yeah, their services are good. I'm just not a...",Google provides online related services and p...,"I'm not a huge fan of Google, but I use it a..."
3,"By the way, do you like Fish?",Yes.Google is the biggest search engine and G...,I like Google Chrome. Do you use it as well f...,"It is very interesting. Google provide ""Chrom...",Did you know Google had hundreds of live goat...,Google is leading the alphabet subsidiary and...,"Yeah, their services are good. I'm just not a...",Google provides online related services and p...
4,Yes. They form a sister group of tourniquets-...,"By the way, do you like Fish?",Yes.Google is the biggest search engine and G...,I like Google Chrome. Do you use it as well f...,"It is very interesting. Google provide ""Chrom...",Did you know Google had hundreds of live goat...,Google is leading the alphabet subsidiary and...,"Yeah, their services are good. I'm just not a..."


In [ ]:
df_trn, df_val = train_test_split(df, test_size = 0.2)

# **CONVERTING THE CONTEXT AND RESPONSE DATAFRAME INTO A SINGLE CONVERSATION STRING**

In [ ]:
#separating conversation strings and telling the model when a person is done speaking
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)
      
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted

def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

# **TRAINING AND EVALUATION**

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
  def main(df_trn, df_val):
    args = Args()

    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [ ]:
main(df_trn, df_val)


02/23/2022 12:27:45 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:807: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
02/23/2022 12:28:14 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7fdc9005c3d0>
02/23/2022 12:28:14 - INFO - __main__ -   Creating features from dataset file at cached
02/23/2022 12:30:10 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTor

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/39994 [00:00<?, ?it/s]

02/23/2022 12:45:24 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-5000
02/23/2022 12:45:38 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-5000
02/23/2022 13:00:54 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-10000
02/23/2022 13:01:06 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-10000
02/23/2022 13:16:24 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-15000
02/23/2022 13:16:36 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-15000
02/23/2022 13:31:55 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-20000
02/23/2022 13:32:07 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-20000
02/23/2022 13:47:26 - INFO - __main__ -   

Iteration:   0%|          | 0/39994 [00:00<?, ?it/s]

...

02/24/2022 02:58:03 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-280000
02/24/2022 02:58:16 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-280000
02/24/2022 03:13:34 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-285000
02/24/2022 03:13:47 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-285000
02/24/2022 03:29:05 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-290000
02/24/2022 03:29:19 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-290000
02/24/2022 03:44:40 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-295000
02/24/2022 03:44:51 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-295000
02/24/2022 04:00:06 - INFO - __m

Iteration:   0%|          | 0/39994 [00:00<?, ?it/s]

02/24/2022 05:01:54 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-320000
02/24/2022 05:02:08 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-320000
02/24/2022 05:17:19 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-325000
02/24/2022 05:17:32 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-325000
02/24/2022 05:32:44 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-330000
02/24/2022 05:32:57 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-330000
02/24/2022 05:48:11 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-335000
02/24/2022 05:48:24 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-335000
02/24/2022 06:03:39 - INFO - __m

Iteration:   0%|          | 0/39994 [00:00<?, ?it/s]

02/24/2022 07:05:31 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-360000
02/24/2022 07:05:45 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-360000
02/24/2022 07:21:04 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-365000
02/24/2022 07:21:17 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-365000
02/24/2022 07:36:38 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-370000
02/24/2022 07:36:49 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-370000
02/24/2022 07:52:06 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-375000
02/24/2022 07:52:20 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-375000
02/24/2022 08:07:36 - INFO - __m

In [ ]:
import transformers
transformers.logging.set_verbosity_info()

# **CONTINUING TRAINING AFTER IT SHUT DOWN**

In [ ]:
main(df_trn, df_val)

02/24/2022 09:56:14 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
loading configuration file https://huggingface.co/microsoft/DialoGPT-medium/resolve/main/config.json from cache at cached/066c0238a1dab50404e7d118e7ad1468d20a1fc18c3f2ad1036366759bfc343d.c26bcfbd792a38251a4fb555d9110e87dcc2ecaee13ac0a027d1584df8a09634
Model config GPT2Config {
  "_name_or_path": "microsoft/DialoGPT-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
 

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/39994 [00:00<?, ?it/s]

Configuration saved in output-medium-50-10-epochs/checkpoint-390000/config.json
Model weights saved in output-medium-50-10-epochs/checkpoint-390000/pytorch_model.bin
tokenizer config file saved in output-medium-50-10-epochs/checkpoint-390000/tokenizer_config.json
Special tokens file saved in output-medium-50-10-epochs/checkpoint-390000/special_tokens_map.json
02/24/2022 10:34:30 - INFO - __main__ -   Saving model checkpoint to output-medium-50-10-epochs/checkpoint-390000
02/24/2022 10:34:41 - INFO - __main__ -   Saving optimizer and scheduler states to output-medium-50-10-epochs/checkpoint-390000
Configuration saved in output-medium-50-10-epochs/checkpoint-395000/config.json
Model weights saved in output-medium-50-10-epochs/checkpoint-395000/pytorch_model.bin
tokenizer config file saved in output-medium-50-10-epochs/checkpoint-395000/tokenizer_config.json
Special tokens file saved in output-medium-50-10-epochs/checkpoint-395000/special_tokens_map.json
02/24/2022 11:01:10 - INFO - __mai

Evaluating:   0%|          | 0/9999 [00:00<?, ?it/s]

02/24/2022 11:37:41 - INFO - __main__ -   ***** Eval results  *****
02/24/2022 11:37:41 - INFO - __main__ -     perplexity = tensor(1.0628)


{'perplexity_': tensor(1.0628)}

# **CHATTING WITH THE MODEL**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
model = AutoModelWithLMHead.from_pretrained('output-medium-50-10-epochs')
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)
# append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
# generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,
        temperature = 0.8
    )
    
    # pretty print last ouput tokens from bot
    print("SeanBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

loading configuration file https://huggingface.co/microsoft/DialoGPT-medium/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/066c0238a1dab50404e7d118e7ad1468d20a1fc18c3f2ad1036366759bfc343d.c26bcfbd792a38251a4fb555d9110e87dcc2ecaee13ac0a027d1584df8a09634
Model config GPT2Config {
  "_name_or_path": "microsoft/DialoGPT-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_us

>> User:Do you like sports?
SeanBot:  I'm not really into sports, but soccer isn't bad. My friend is obsessed with it and was telling me about this guy Freddy Adu who became pro at 14, I think
>> User:What do you like doing since you don't like sports?
SeanBot:  Well I used to play a lot of baseball but I don't play anymore, what about you?
>> User:I run.
SeanBot:  That's cool. Do you know why baseball managers wear uniforms?
>> User:I do.
SeanBot:  Are you kidding? I wonder if they do it so they don't get fined!
>> User:They do it because they have to.
SeanBot:  Wow, that's pretty cool. Can you believe the Ottoman Empire was still in existence when the Cubs first won the World Series?
